In [1]:
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

In [2]:
# Đường dẫn đến file json chứa thông tin xác thực
credentials_path = 'key.json'

# Khởi tạo phiên làm việc với Google Sheets
credentials = Credentials.from_service_account_file(credentials_path, scopes=[
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
])
gc = gspread.authorize(credentials)

In [3]:
# Tạo hai thư mục trên Google Drive để chứa dữ liệu trước và sau gộp
old_folder_id = '1zuHsJ1bFvl2nxlMYvXw_QP8sHbeyeobt'
new_folder_id = '1PrpzFgSwS4eqiyENPWKpeuWcbbXpR4GN'
target_sheet_name = 'final_data'
target_sheet = gc.create(target_sheet_name)
target_sheet_id = target_sheet.id
drive_service = build('drive', 'v3', credentials=credentials)

# Kiểm tra xem tệp tin có tồn tại ở thư mục gốc không?
try:
    file_info = drive_service.files().get(fileId=target_sheet_id, fields='parents').execute()
except Exception as e:
    print(f"Lỗi khi lấy thông tin tệp tin: {e}")

# Xóa tệp tin từ thư mục gốc nếu nó tồn tại
if 'parents' in file_info:
    try:
        drive_service.files().update(fileId=target_sheet_id, removeParents=old_folder_id).execute()
        print("Đã xóa thành công tệp tin khỏi thư mục gốc.")
    except Exception as e:
        print(f"Lỗi xóa tệp tin khỏi thư mục gốc: {e}")
else:
    print("Tệp tin không tồn tại, bỏ qua việc xóa khỏi thư mục gốc.")

# Thêm tệp tin vào thư mục mới
try:
    drive_service.files().update(fileId=target_sheet_id, addParents=new_folder_id).execute()
    print("Đã thêm tệp tin vào thư mục mới.")
except Exception as e:
    print(f"Lỗi thêm tệp tin vào thư mục mới: {e}")

# Gộp dữ liệu từ các sheet vào chung một sheet vừa tạo
sheet_names = ["data_2021", "data_2022", "data_2023"]

for sheet_name in sheet_names:
    source_sheet = gc.open(sheet_name).sheet1
    target_data = source_sheet.get_all_values()

    try:
        target_worksheet = target_sheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        target_worksheet = target_sheet.add_worksheet(sheet_name, rows=len(target_data), cols=len(target_data[0]))

    target_worksheet.update('A1', target_data)

print(f"Dữ liệu đã được gộp thành công {target_sheet.url}")

Đã xóa thành công tệp tin khỏi thư mục gốc.
Đã thêm tệp tin vào thư mục mới.


C:\Users\fna\AppData\Local\Temp\ipykernel_2560\152736752.py:44: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  target_worksheet.update('A1', target_data)


Dữ liệu đã được gộp thành công https://docs.google.com/spreadsheets/d/1z-eAvFC5g_425HfZJUwUxB6sKLFA7iHiruZN653jYbM
